In [1]:
import pandas as pd 
import re


In [2]:
full_df = pd.read_csv('lyric_df.csv')
lyrics_df = pd.read_csv('english_lyrics.csv')


In [3]:
full_df

,Unnamed: 0,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,...,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,...,-4.718,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과..."
1,1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,...,-3.715,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and..."
2,2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,...,-4.280,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장..."
3,3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,...,-7.470,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my..."
4,4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,...,-7.381,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,...,-6.113,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...
454,454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,...,-6.449,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing
455,455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,...,-5.406,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya..."
456,456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,...,-7.769,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni..."


In [4]:
full_df = full_df.drop(['Unnamed: 0'], axis=1)

In [5]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,-4.718,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과..."
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,-3.715,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and..."
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,-4.280,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장..."
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,-7.470,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my..."
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,-7.381,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,-6.113,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,-6.449,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,-5.406,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya..."
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,-7.769,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni..."


# Clean full_df lyrics 

In [6]:
lyrics_df

,Unnamed: 0,artist,lyrics,filterd_lyrics
0,0,BIBI,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",...
1,1,BIBI,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cub...
2,2,BIBI,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",cham...
3,3,BIBI,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my...
4,4,BIBI,missing,missing
...,...,...,...,...
453,453,Warren Hue,[Verse 1: Warren Hue](I wanna be like)I've bee...,Verse Warren Hue I wanna be like I ve bee...
454,454,Warren Hue,missing,missing
455,455,Warren Hue,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT tu...
456,456,Warren Hue,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tr...


In [7]:
def remove_song_structure(lyrics):
    # Use a regular expression to remove the specific song structure elements
    lyrics = re.sub(r'\[(Pre-Chorus|Verse 1|Chorus|Intro|Verse|Verse 2|Outro|Bridge)\]', ' ', lyrics)
    
    return lyrics

In [8]:
# Create a new column 'col_english' with the modified values
lyrics_df['final_lyrics'] = lyrics_df['filterd_lyrics'].apply(remove_song_structure)

In [9]:
lyrics_df = lyrics_df.drop(['Unnamed: 0'], axis=1)

In [10]:
lyrics_df 

,artist,lyrics,filterd_lyrics,final_lyrics
0,BIBI,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",...,...
1,BIBI,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cub...,Dice and Ice cub...
2,BIBI,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",cham...,cham...
3,BIBI,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my...,BAD SAD AND MAD Refrain You blued my...
4,BIBI,missing,missing,missing
...,...,...,...,...
453,Warren Hue,[Verse 1: Warren Hue](I wanna be like)I've bee...,Verse Warren Hue I wanna be like I ve bee...,Verse Warren Hue I wanna be like I ve bee...
454,Warren Hue,missing,missing,missing
455,Warren Hue,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT tu...,Uh uh ayy ya yaUh uh turn turn ya yaT tu...
456,Warren Hue,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tr...,Won t you run away with me this evening Oh tr...


In [11]:
missing_songs = lyrics_df.loc[lyrics_df['final_lyrics'] == 'missing']

In [12]:
missing_songs.reset_index(inplace=True)

In [13]:
missing_songs

,index,artist,lyrics,filterd_lyrics,final_lyrics
0,4,BIBI,missing,missing,missing
1,9,BIBI,missing,missing,missing
2,11,BIBI,missing,missing,missing
3,12,BIBI,missing,missing,missing
4,13,BIBI,missing,missing,missing
...,...,...,...,...,...
172,444,Warren Hue,missing,missing,missing
173,446,Rich Brian,missing,missing,missing
174,448,Warren Hue,missing,missing,missing
175,452,Warren Hue,missing,missing,missing


In [ ]:
missing_songs = missing_songs.drop(['Unnamed: 0'], axis=1)

In [15]:
missing_songs = missing_songs.drop(['index'], axis=1)

In [16]:
missing_songs

,artist,lyrics,filterd_lyrics,final_lyrics
0,BIBI,missing,missing,missing
1,BIBI,missing,missing,missing
2,BIBI,missing,missing,missing
3,BIBI,missing,missing,missing
4,BIBI,missing,missing,missing
...,...,...,...,...
172,Warren Hue,missing,missing,missing
173,Rich Brian,missing,missing,missing
174,Warren Hue,missing,missing,missing
175,Warren Hue,missing,missing,missing


In [17]:
songs_df = lyrics_df.loc[lyrics_df['final_lyrics'] != 'missing']

In [18]:
songs_df

,artist,lyrics,filterd_lyrics,final_lyrics
0,BIBI,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",...,...
1,BIBI,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cub...,Dice and Ice cub...
2,BIBI,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",cham...,cham...
3,BIBI,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my...,BAD SAD AND MAD Refrain You blued my...
5,BIBI,"[비비 ""신경쓰여"" 가사][Verse 1]불 꺼진 방안 뒹굴다전화는 왜 잠만 잘 자...",...,...
...,...,...,...,...
451,Warren Hue,"[Verse 1]Ayy, ayy, ayyWent to Los AngelesDissi...",Ayy ayy ayyWent to Los AngelesDissin all th...,Ayy ayy ayyWent to Los AngelesDissin all th...
453,Warren Hue,[Verse 1: Warren Hue](I wanna be like)I've bee...,Verse Warren Hue I wanna be like I ve bee...,Verse Warren Hue I wanna be like I ve bee...
455,Warren Hue,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT tu...,Uh uh ayy ya yaUh uh turn turn ya yaT tu...
456,Warren Hue,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tr...,Won t you run away with me this evening Oh tr...


In [19]:
def remove_extra_spaces(text):
    # Use a regular expression to remove multiple consecutive spaces
    text = re.sub(r' {2,}', ' ', text)
    
    return text

In [24]:
lyrics_df ['Processed_lyrics'] = lyrics_df ['final_lyrics'].apply(remove_extra_spaces)

In [25]:
lyrics_df

,artist,lyrics,filterd_lyrics,final_lyrics,Processed_lyrics
0,BIBI,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",...,...,Watch me danceWatch me sing Watch me danceWat...
1,BIBI,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cub...,Dice and Ice cub...,Dice and Ice cube Home girls bitches Spin it ...
2,BIBI,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",cham...,cham...,champagne It might be pretty it might be beau...
3,BIBI,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my...,BAD SAD AND MAD Refrain You blued my...,BAD SAD AND MAD Refrain You blued my bruisePi...
4,BIBI,missing,missing,missing,missing
...,...,...,...,...,...
453,Warren Hue,[Verse 1: Warren Hue](I wanna be like)I've bee...,Verse Warren Hue I wanna be like I ve bee...,Verse Warren Hue I wanna be like I ve bee...,Verse Warren Hue I wanna be like I ve been st...
454,Warren Hue,missing,missing,missing,missing
455,Warren Hue,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT tu...,Uh uh ayy ya yaUh uh turn turn ya yaT tu...,Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...
456,Warren Hue,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tr...,Won t you run away with me this evening Oh tr...,Won t you run away with me this evening Oh tru...


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By

/var/folders/06/dlzbbxfn6tgfd6s50hkzfk3r0000gn/T/ipykernel_69949/2402906827.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/og/Downloads/chromedriver")


In [22]:
#function to scrape lyrics from genius
def scrape_genres(full_df):
    # Set up a webdriver object to control a Chrome browser.
    driver = webdriver.Chrome(executable_path="/Users/og/Desktop/chromedriver")   
    for index, row in full_df.iterrows():
        artistname2 = row['artist'].replace(' ','-') 
        songname2 = row['track'].replace(' ','-')
        # page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
        # html = BeautifulSoup(page.text, 'html.parser')
        # lyrics1 = html.find("div", class_="lyrics")
        # lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-6 YYrds") 
        driver.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
        element = driver.find_element(By.XPATH, "//main/div[3]/div[3]/div[1]/div[5]/div[1]/div/div[5]/a")
        tag = element.get_attribute("text")
        print(tage)
        # if lyrics1:
        #     lyrics = lyrics1.get_text()
        # elif lyrics2:
        #     lyrics = lyrics2.get_text()
        # elif lyrics1 == lyrics2 == None:
        #     lyrics = None
        # test = lyrics
        full_df.loc[index, 'genres'] = tag
    return full_df


In [ ]:
final = scrape_genres(full_df)